In [ ]:
!pip install ultralytics
!pip install albumentations
!pip install torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Paths (update these with your folder structure in Google Drive)
data_dir = '/content/drive/MyDrive/XYZ/labeled/images'  # Folder containing original images
annotations_dir = '/content/drive/MyDrive/XYZ/labeled/labels'  # Folder containing annotation text files
augmented_dir = '/content/drive/MyDrive/XYZ/augmented_data'  # Folder to save augmented images and annotations

os.makedirs(augmented_dir, exist_ok=True)

# Augmentation pipeline
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.GaussianBlur(p=0.2),
    A.HueSaturationValue(p=0.3),
    A.CLAHE(p=0.2),
    A.RandomCrop(width=256, height=256, p=0.5),
    ToTensorV2()
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

# Function to apply augmentation
def augment_image(image_path, annotation_path, save_dir, aug_count=5):
    image = cv2.imread(image_path)
    h, w = image.shape[:2]

    with open(annotation_path, 'r') as f:
        annotations = f.readlines()

    bboxes = []
    class_labels = []
    for ann in annotations:
        parts = ann.strip().split()
        class_id = int(parts[0])
        bbox = list(map(float, parts[1:5]))  # [x_center, y_center, width, height]
        class_labels.append(class_id)
        bboxes.append(bbox)

    for i in range(aug_count):
        augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)
        aug_image = augmented['image'].permute(1, 2, 0).numpy()
        aug_bboxes = augmented['bboxes']
        aug_labels = augmented['class_labels']

        aug_image_name = os.path.splitext(os.path.basename(image_path))[0] + f'_aug{i}.jpg'
        aug_annotation_name = os.path.splitext(os.path.basename(annotation_path))[0] + f'_aug{i}.txt'

        cv2.imwrite(os.path.join(save_dir, aug_image_name), aug_image)

        with open(os.path.join(save_dir, aug_annotation_name), 'w') as f:
            for bbox, label in zip(aug_bboxes, aug_labels):
                f.write(f"{label} {' '.join(map(str, bbox))}\n")

# Apply augmentation to all images
for img_file in os.listdir(data_dir):
    if img_file.endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(data_dir, img_file)
        ann_path = os.path.join(annotations_dir, os.path.splitext(img_file)[0] + '.txt')

        if os.path.exists(ann_path):
            augment_image(img_path, ann_path, augmented_dir, aug_count=3)  # 5 augmentations per image

Mounted at /content/drive


/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:58: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


In [ ]:
import os
import shutil

# Define main directory (Change this path if needed)
main_dir = '/content/drive/MyDrive/XYZ/augmented_data'  # Update with the actual path
images_dir = os.path.join(main_dir, 'images')
labels_dir = os.path.join(main_dir, 'labels')

# Create folders if they don't exist
os.makedirs(images_dir, exist_ok=True)
os.makedirs(labels_dir, exist_ok=True)

# Loop through files in the main directory
for file in os.listdir(main_dir):
    file_path = os.path.join(main_dir, file)

    # Check if it's a file
    if os.path.isfile(file_path):
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff')):
            shutil.move(file_path, images_dir)  # Move image files
        elif file.lower().endswith('.txt'):
            shutil.move(file_path, labels_dir)  # Move annotation files

print("Files have been organized into 'images' and 'labels' folders.")

Files have been organized into 'images' and 'labels' folders.


In [ ]:
data_yaml = """
train: /content/drive/MyDrive/XYZ/augmented_data/images
val: /content/drive/MyDrive/XYZ/test/images

nc: 2
names: ['weed', 'crops']
"""

with open('/content/drive/MyDrive/XYZ/data.yaml', 'w') as f:
    f.write(data_yaml)

In [ ]:
import os

# Paths to the train directory
train_image_dir = '/content/drive/MyDrive/XYZ/augmented_data/images'
train_label_dir = '/content/drive/MyDrive/XYZ/augmented_data/labels'

# Count images and labels
num_images = len([f for f in os.listdir(train_image_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])
num_labels = len([f for f in os.listdir(train_label_dir) if f.endswith('.txt')])

# Display the counts
print(f"Total Images in Train Directory: {num_images}")
print(f"Total Annotations in Train Directory: {num_labels}")

Total Images in Train Directory: 600
Total Annotations in Train Directory: 600


In [ ]:
from ultralytics import YOLO
from PIL import Image
import torchvision.transforms as transforms
import os
from torch.utils.data import Dataset, DataLoader

# Custom Dataset Class
class YOLODataset(Dataset):
    def __init__(self, images_folder, labels_folder, transform=None):
        self.images_folder = images_folder
        self.labels_folder = labels_folder
        self.transform = transform
        self.image_files = [f for f in os.listdir(images_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_file = self.image_files[idx]
        image_path = os.path.join(self.images_folder, image_file)
        label_path = os.path.join(self.labels_folder, os.path.splitext(image_file)[0] + '.txt')

        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        labels = []
        if os.path.exists(label_path):
            with open(label_path, 'r') as label_file:
                labels = label_file.read()

        return image, labels

# Image preprocessing
transform = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Paths to images and labels
images_folder = '/content/drive/MyDrive/XYZ/augmented_data/images'
labels_folder = '/content/drive/MyDrive/XYZ/augmented_data/labels'

# Dataset and DataLoader
dataset = YOLODataset(images_folder, labels_folder, transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)

# Check for the latest checkpoint
checkpoint_dir = '/content/drive/MyDrive/XYZ/runs/detect/train/weights'
last_checkpoint = None

if os.path.exists(checkpoint_dir):
    checkpoints = [f for f in os.listdir(checkpoint_dir) if f.endswith('.pt')]
    if checkpoints:
        last_checkpoint = max(checkpoints, key=lambda x: os.path.getmtime(os.path.join(checkpoint_dir, x)))

# Load the model
if last_checkpoint:
    print(f"Resuming training from checkpoint: {last_checkpoint}")
    model = YOLO(os.path.join(checkpoint_dir, last_checkpoint))
else:
    print("Starting training from scratch")
    model = YOLO('yolov8m.pt')

# Training with early stopping
model.train(
    data='/content/drive/MyDrive/XYZ/data.yaml',  # Replace with your YAML file path
    epochs=100,                                    # Increased epochs
    imgsz=640,
    batch=16,
    patience=20,                                 # Early stopping patience set to 200 epochs
    optimizer='Adam',
    lr0=0.01,
    weight_decay=0.0005,
    augment=True,
    device='0',                                   # Change to 'cpu' if GPU is not available
    resume=bool(last_checkpoint)                  # Resume training if checkpoint exists
)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Starting training from scratch
Ultralytics 8.3.71 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/drive/MyDrive/XYZ/data.yaml, epochs=100, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_l

train: Scanning /content/drive/MyDrive/XYZ/augmented_data/labels.cache... 600 images, 0 backgrounds, 0 corrupt: 100%|██████████| 600/600 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/MyDrive/XYZ/test/labels.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]


...

                   all         50         74       0.77      0.815      0.855      0.516



100 epochs completed in 0.669 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 52.0MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics 8.3.71 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 218 layers, 25,840,918 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.30s/it]


                   all         50         74      0.844      0.795      0.877      0.552
                  weed         24         42      0.882       0.81      0.903      0.584
                 crops         26         32      0.806       0.78       0.85      0.519
Speed: 0.2ms preprocess, 41.3ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/train3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x793c377b9650>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [ ]:
model.save('/content/drive/MyDrive/XYZ/yolov8_trained_model_final.pt')

In [ ]:
from ultralytics import YOLO
import torch
import os
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

# Load the trained YOLOv8 model
model = YOLO('/content/drive/MyDrive/XYZ/yolov8_trained_model_final.pt')

# Paths to test images and annotations
test_image_dir = '/content/drive/MyDrive/XYZ/test/images'
test_label_dir = '/content/drive/MyDrive/XYZ/test/labels'

# Function to load ground truth annotations
def load_ground_truth(label_path):
    with open(label_path, 'r') as file:
        lines = file.readlines()
    labels = []
    for line in lines:
        parts = list(map(float, line.strip().split()))
        labels.append(parts)  # [class_id, x_center, y_center, width, height]
    return np.array(labels)

# Evaluation metrics initialization
all_true_classes = []
all_pred_classes = []
all_iou_scores = []

# IoU calculation function
def calculate_iou(box1, box2):
    # Convert from (x_center, y_center, w, h) to (x1, y1, x2, y2)
    def xywh_to_xyxy(box):
        x_c, y_c, w, h = box
        return [x_c - w / 2, y_c - h / 2, x_c + w / 2, y_c + h / 2]

    box1 = xywh_to_xyxy(box1)
    box2 = xywh_to_xyxy(box2)

    # Intersection coordinates
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    # Intersection area
    intersection = max(0, x2 - x1) * max(0, y2 - y1)

    # Union area
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = box1_area + box2_area - intersection

    # IoU
    return intersection / union if union != 0 else 0

# Inference and evaluation loop
for image_file in os.listdir(test_image_dir):
    if image_file.endswith(('.jpg', '.png')):
        image_path = os.path.join(test_image_dir, image_file)
        label_path = os.path.join(test_label_dir, os.path.splitext(image_file)[0] + '.txt')

        # Skip if no corresponding annotation
        if not os.path.exists(label_path):
            continue

        # Inference
        results = model(image_path)
        predictions = results[0].boxes  # Bounding boxes

        # Ground truth
        ground_truths = load_ground_truth(label_path)

        # Get original image dimensions
        image_width, image_height = results[0].orig_shape[1], results[0].orig_shape[0]

        # Matching predictions with ground truth
        for pred in predictions:
            pred_class = int(pred.cls.item())
            pred_box = pred.xywh[0].cpu().numpy()

            # Normalize predicted box coordinates
            pred_box_normalized = pred_box / [image_width, image_height, image_width, image_height]

            iou_scores = [calculate_iou(pred_box_normalized, gt[1:]) for gt in ground_truths]
            best_match_idx = np.argmax(iou_scores)

            all_true_classes.append(int(ground_truths[best_match_idx][0]))
            all_pred_classes.append(pred_class)
            all_iou_scores.append(iou_scores[best_match_idx])

# Metrics Calculation
precision = precision_score(all_true_classes, all_pred_classes, average='weighted')
recall = recall_score(all_true_classes, all_pred_classes, average='weighted')
f1 = f1_score(all_true_classes, all_pred_classes, average='weighted')
mean_iou = np.mean(all_iou_scores)

# Calculate mAP@[.5:.95]
map_results = model.val()
map_50_95 = map_results.box.map  # Mean Average Precision @[.5:.95]

# Custom Evaluation Metric
custom_metric = 0.5 * f1 + 0.5 * map_50_95

# Display Metrics
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Mean IoU: {mean_iou:.4f}")
print(f"mAP@[.5:.95]: {map_50_95:.4f}")
print(f"Custom Evaluation Metric: {custom_metric:.4f}")


image 1/1 /content/drive/MyDrive/XYZ/test/images/agri_0_9279_jpeg.rf.1e754c96a16dc1e519a2dc7ef32a4eb7.jpg: 640x640 2 cropss, 36.3ms
Speed: 4.5ms preprocess, 36.3ms inference, 3.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/XYZ/test/images/agri_0_9532_jpeg.rf.cec78142a3ad2cbd2466c222131f846a.jpg: 640x640 1 crops, 37.2ms
Speed: 4.6ms preprocess, 37.2ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/XYZ/test/images/agri_0_9451_jpeg.rf.a0e85f372406c58574307c4397cf3657.jpg: 640x640 3 weeds, 1 crops, 36.4ms
Speed: 4.5ms preprocess, 36.4ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/XYZ/test/images/agri_0_9424_jpeg.rf.f758222e980e6d4265312a3cf6ac6751.jpg: 640x640 2 weeds, 36.4ms
Speed: 7.8ms preprocess, 36.4ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/XYZ/test/images/agri_0_9698_jpeg.rf.2d42

val: Scanning /content/drive/MyDrive/XYZ/test/labels.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:04<00:00,  1.09s/it]


                   all         50         74      0.753      0.834      0.847      0.521
                  weed         24         42      0.741      0.886      0.885      0.548
                 crops         26         32      0.766      0.781      0.809      0.494
Speed: 0.4ms preprocess, 42.7ms inference, 0.0ms loss, 8.1ms postprocess per image
Results saved to runs/detect/val
Precision: 0.9545
Recall: 0.9545
F1 Score: 0.9545
Mean IoU: 0.6839
mAP@[.5:.95]: 0.5210
Custom Evaluation Metric: 0.7378


In [ ]:
import os
import shutil
from ultralytics import YOLO
from PIL import Image

# Paths
labeled_images_folder = '/content/drive/MyDrive/XYZ/augmented_data/images'
labeled_labels_folder = '/content/drive/MyDrive/XYZ/augmented_data/labels'
unlabeled_images_folder = '/content/drive/MyDrive/XYZ/unlabeled'
new_train_images_folder = '/content/drive/MyDrive/XYZ/train/images'
new_train_labels_folder = '/content/drive/MyDrive/XYZ/train/labels'

# Create new train directories
os.makedirs(new_train_images_folder, exist_ok=True)
os.makedirs(new_train_labels_folder, exist_ok=True)

# Copy labeled data to the new train directory
for file_name in os.listdir(labeled_images_folder):
    if file_name.endswith(('.png', '.jpg', '.jpeg')):
        shutil.copy(os.path.join(labeled_images_folder, file_name), new_train_images_folder)
        label_file = os.path.splitext(file_name)[0] + '.txt'
        shutil.copy(os.path.join(labeled_labels_folder, label_file), new_train_labels_folder)

# Load the trained YOLO model
model_path = '/content/drive/MyDrive/XYZ/yolov8_trained_model_final.pt'  # Adjust if needed
model = YOLO(model_path)


confidence_threshold_image = 0.8  # At least one detection above this threshold to include the image
confidence_threshold_detection = 0.8  # Only detections with confidence above this threshold are considered

for img_file in os.listdir(unlabeled_images_folder):
    if img_file.endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(unlabeled_images_folder, img_file)
        results = model(img_path)

        # Initialize list to collect pseudo-labels for the current image
        pseudo_labels = []
        image_included = False  # Flag to track if the image should be included based on detection threshold

        for result in results:
            for box in result.boxes:
                confidence = box.conf.item()
                cls = int(box.cls.item())
                xywh = box.xywh[0].tolist()  # [x_center, y_center, width, height]

                # Print confidence score
                print(f"Image: {img_file}, Class: {cls}, Confidence: {confidence:.4f}")

                # Check if the detection meets the detection threshold
                if confidence >= confidence_threshold_detection:
                  pseudo_labels.append(f"{cls} {' '.join(map(str, xywh))}")
                  image_included = True  # Include image if any valid detection is found

        # Only save image and annotation if valid detections exist
        if image_included and pseudo_labels:
            # Copy the image file to the new directory
            shutil.copy(img_path, new_train_images_folder)

            # Write the annotations to the corresponding annotation file
            label_file_path = os.path.join(new_train_labels_folder, os.path.splitext(img_file)[0] + '.txt')
            with open(label_file_path, 'w') as label_file:
                label_file.write('\n'.join(pseudo_labels))


Streaming output truncated to the last 5000 lines.
Image: agri_0_1028_jpeg.rf.32fc950fe95beacd547893f9915ab5a9.jpg, Class: 0, Confidence: 0.2653
Image: agri_0_1028_jpeg.rf.32fc950fe95beacd547893f9915ab5a9.jpg, Class: 0, Confidence: 0.2612
Image: agri_0_1028_jpeg.rf.32fc950fe95beacd547893f9915ab5a9.jpg, Class: 0, Confidence: 0.2503

image 1/1 /content/drive/MyDrive/XYZ/unlabeled/agri_0_1017_jpeg.rf.10421009c95805e81d82a2b6a8c3b116.jpg: 640x640 4 cropss, 17.8ms
Speed: 8.9ms preprocess, 17.8ms inference, 9.7ms postprocess per image at shape (1, 3, 640, 640)
Image: agri_0_1017_jpeg.rf.10421009c95805e81d82a2b6a8c3b116.jpg, Class: 1, Confidence: 0.7520
Image: agri_0_1017_jpeg.rf.10421009c95805e81d82a2b6a8c3b116.jpg, Class: 1, Confidence: 0.4331
Image: agri_0_1017_jpeg.rf.10421009c95805e81d82a2b6a8c3b116.jpg, Class: 1, Confidence: 0.3920
Image: agri_0_1017_jpeg.rf.10421009c95805e81d82a2b6a8c3b116.jpg, Class: 1, Confidence: 0.3026

image 1/1 /content/drive/MyDrive/XYZ/unlabeled/agri_0_1018_jpe

In [16]:
import os

# Paths to the train directory
train_image_dir = '/content/drive/MyDrive/XYZ/train/images'
train_label_dir = '/content/drive/MyDrive/XYZ/train/labels'

# Count images and labels
num_images = len([f for f in os.listdir(train_image_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])
num_labels = len([f for f in os.listdir(train_label_dir) if f.endswith('.txt')])

# Display the counts
print(f"Total Images in Train Directory: {num_images}")
print(f"Total Annotations in Train Directory: {num_labels}")

Total Images in Train Directory: 980
Total Annotations in Train Directory: 980


In [ ]:
import os
import shutil
from ultralytics import YOLO
from PIL import Image

# Paths
unlabeled_images_folder = '/content/drive/MyDrive/XYZ/unlabeled'  # Folder with unlabeled images
new_train_images_folder = '/content/drive/MyDrive/XYZ/train/images'  # Folder to store pseudo-labeled images
new_train_labels_folder = '/content/drive/MyDrive/XYZ/train/labels'  # Folder to store pseudo-labels

# Load the trained YOLO model (the one you just trained)
model_path = '/content/drive/MyDrive/XYZ/yolov8_trained_model_final_2.pt'  # Adjust if needed
model = YOLO(model_path)

# Confidence threshold for detection
confidence_threshold_detection = 0.8  # Only detections with confidence above this threshold are considered

# Create directories if they don't exist
os.makedirs(new_train_images_folder, exist_ok=True)
os.makedirs(new_train_labels_folder, exist_ok=True)

# Process unlabeled images
for img_file in os.listdir(unlabeled_images_folder):
    if img_file.endswith(('.png', '.jpg', '.jpeg')):  # Only process image files
        img_path = os.path.join(unlabeled_images_folder, img_file)

        # Load the image to get its dimensions for normalization
        img = Image.open(img_path)
        img_width, img_height = img.size

        # Run inference on the image
        results = model(img_path)  # Run inference on the image

        # Initialize list to collect pseudo-labels for the current image
        pseudo_labels = []
        image_included = False  # Flag to track if the image should be included based on detection threshold

        for result in results:
            for box in result.boxes:
                confidence = box.conf.item()
                cls = int(box.cls.item())
                xywh = box.xywh[0].tolist()  # [x_center, y_center, width, height]

                # Normalize the coordinates
                x_center_norm = xywh[0] / img_width
                y_center_norm = xywh[1] / img_height
                width_norm = xywh[2] / img_width
                height_norm = xywh[3] / img_height

                # Print confidence score for each detection
                print(f"Image: {img_file}, Class: {cls}, Confidence: {confidence:.4f}")

                # Check if the detection meets the confidence threshold
                if confidence >= confidence_threshold_detection:
                    # Save the normalized pseudo-label in YOLO format
                    pseudo_labels.append(f"{cls} {x_center_norm} {y_center_norm} {width_norm} {height_norm}")
                    image_included = True  # Mark the image for inclusion if any valid detections are found

        # Only save the image and annotations if valid detections exist
        if image_included and pseudo_labels:
            # Copy the image to the new directory
            shutil.copy(img_path, new_train_images_folder)

            # Write the annotations to the corresponding annotation file
            label_file_path = os.path.join(new_train_labels_folder, os.path.splitext(img_file)[0] + '.txt')
            with open(label_file_path, 'w') as label_file:
                label_file.write('\n'.join(pseudo_labels))  # Save pseudo-labels for the detections



image 1/1 /content/drive/MyDrive/XYZ/unlabeled/agri_0_1509_jpeg.rf.0d24077902e5229abad25227f9b58df9.jpg: 640x640 (no detections), 16.5ms
Speed: 4.1ms preprocess, 16.5ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/XYZ/unlabeled/agri_0_7839_jpeg.rf.54cce7e93ac61d0bba4cc447e6d1b149.jpg: 640x640 (no detections), 16.4ms
Speed: 3.1ms preprocess, 16.4ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/XYZ/unlabeled/agri_0_8105_jpeg.rf.fe71fa27f32018eb82f7e398495b0c8d.jpg: 640x640 1 crops, 16.8ms
Speed: 5.2ms preprocess, 16.8ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 640)
Image: agri_0_8105_jpeg.rf.fe71fa27f32018eb82f7e398495b0c8d.jpg, Class: 1, Confidence: 0.5304

image 1/1 /content/drive/MyDrive/XYZ/unlabeled/agri_0_6851_jpeg.rf.80c38e10f43fc208c64224004507c878.jpg: 640x640 1 crops, 16.4ms
Speed: 4.7ms preprocess, 16.4ms inference, 1.8ms postprocess per image at shape

In [ ]:
from ultralytics import YOLO
import os

# Path to the YAML file
new_data_yaml = '/content/drive/MyDrive/XYZ/train/data_2.yaml'

# Path to save models
save_folder = '/content/drive/MyDrive/XYZ/saved_models'

# Create the folder if it doesn't exist
os.makedirs(save_folder, exist_ok=True)

# Initialize the model
model = YOLO('yolov8s.pt')

# Train the model on your data
model.train(
    data=new_data_yaml,                            # New YAML file
    epochs=75,                                     # Number of epochs
    imgsz=640,                                     # Image size
    batch=16,                                      # Batch size
    optimizer='Adam',                              # Optimizer
    lr0=0.005,                                     # Initial learning rate
    weight_decay=0.0005,                           # Weight decay
    augment=True,                                  # Data augmentation
    device='0',                                    # GPU device
    save_period=10,                                 # Save the model every 5 epochs
    save_dir=save_folder                           # Custom save directory
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 21.5M/21.5M [00:00<00:00, 38.9MB/s]


Ultralytics 8.3.71 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/XYZ/train/data_2.yaml, epochs=75, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=10, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tr

100%|██████████| 755k/755k [00:00<00:00, 117MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

...

                   all         50         74      0.946      0.377      0.527       0.36



75 epochs completed in 0.573 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 22.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.71 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.07it/s]


                   all         50         74      0.972      0.401      0.606      0.391
                  weed         24         42      0.945      0.595      0.698      0.465
                 crops         26         32          1      0.208      0.514      0.316
Speed: 0.2ms preprocess, 26.0ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7da8aad4cbd0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [ ]:
model.save('/content/drive/MyDrive/XYZ/yolov8_trained_model_final_2.pt')

In [19]:
data_yaml = """
train: /content/drive/MyDrive/XYZ/train/images
val: /content/drive/MyDrive/XYZ/test/images

nc: 2
names: ['weed', 'crops']
"""

with open('/content/drive/MyDrive/XYZ/data_4.yaml', 'w') as f:
    f.write(data_yaml)

In [ ]:
from ultralytics import YOLO
import torch
import os
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

# Load the trained YOLOv8 model
model = YOLO('/content/drive/MyDrive/XYZ/yolov8_trained_model_final_2.pt')

# Paths to test images and annotations
test_image_dir = '/content/drive/MyDrive/XYZ/test/images'
test_label_dir = '/content/drive/MyDrive/XYZ/test/labels'

# Function to load ground truth annotations
def load_ground_truth(label_path):
    with open(label_path, 'r') as file:
        lines = file.readlines()
    labels = []
    for line in lines:
        parts = list(map(float, line.strip().split()))
        labels.append(parts)  # [class_id, x_center, y_center, width, height]
    return np.array(labels)

# Evaluation metrics initialization
all_true_classes = []
all_pred_classes = []
all_iou_scores = []

# IoU calculation function
def calculate_iou(box1, box2):
    # Convert from (x_center, y_center, w, h) to (x1, y1, x2, y2)
    def xywh_to_xyxy(box):
        x_c, y_c, w, h = box
        return [x_c - w / 2, y_c - h / 2, x_c + w / 2, y_c + h / 2]

    box1 = xywh_to_xyxy(box1)
    box2 = xywh_to_xyxy(box2)

    # Intersection coordinates
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    # Intersection area
    intersection = max(0, x2 - x1) * max(0, y2 - y1)

    # Union area
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = box1_area + box2_area - intersection

    # IoU
    return intersection / union if union != 0 else 0

# Inference and evaluation loop
for image_file in os.listdir(test_image_dir):
    if image_file.endswith(('.jpg', '.png')):
        image_path = os.path.join(test_image_dir, image_file)
        label_path = os.path.join(test_label_dir, os.path.splitext(image_file)[0] + '.txt')

        # Skip if no corresponding annotation
        if not os.path.exists(label_path):
            continue

        # Inference
        results = model(image_path)
        predictions = results[0].boxes  # Bounding boxes

        # Ground truth
        ground_truths = load_ground_truth(label_path)

        # Get original image dimensions
        image_width, image_height = results[0].orig_shape[1], results[0].orig_shape[0]

        # Matching predictions with ground truth
        for pred in predictions:
            pred_class = int(pred.cls.item())
            pred_box = pred.xywh[0].cpu().numpy()

            # Normalize predicted box coordinates
            pred_box_normalized = pred_box / [image_width, image_height, image_width, image_height]

            iou_scores = [calculate_iou(pred_box_normalized, gt[1:]) for gt in ground_truths]
            best_match_idx = np.argmax(iou_scores)

            all_true_classes.append(int(ground_truths[best_match_idx][0]))
            all_pred_classes.append(pred_class)
            all_iou_scores.append(iou_scores[best_match_idx])

# Metrics Calculation
precision = precision_score(all_true_classes, all_pred_classes, average='weighted')
recall = recall_score(all_true_classes, all_pred_classes, average='weighted')
f1 = f1_score(all_true_classes, all_pred_classes, average='weighted')
mean_iou = np.mean(all_iou_scores)

# Calculate mAP@[.5:.95]
map_results = model.val()
map_50_95 = map_results.box.map  # Mean Average Precision @[.5:.95]

# Custom Evaluation Metric
custom_metric = 0.5 * f1 + 0.5 * map_50_95

# Display Metrics
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Mean IoU: {mean_iou:.4f}")
print(f"mAP@[.5:.95]: {map_50_95:.4f}")
print(f"Custom Evaluation Metric: {custom_metric:.4f}")


image 1/1 /content/drive/MyDrive/XYZ/test/images/agri_0_9135_jpeg.rf.f5afbc71d5606255b3522bfc6feec990.jpg: 640x640 (no detections), 16.4ms
Speed: 9.4ms preprocess, 16.4ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/XYZ/test/images/agri_0_8991_jpeg.rf.e46eb3e83c49f7b83fd8b49300ff6bf6.jpg: 640x640 (no detections), 16.3ms
Speed: 4.7ms preprocess, 16.3ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/XYZ/test/images/agri_0_9116_jpeg.rf.812b125d18a8e7b60a363dc022354c92.jpg: 640x640 1 crops, 16.3ms
Speed: 5.0ms preprocess, 16.3ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/XYZ/test/images/agri_0_9424_jpeg.rf.f758222e980e6d4265312a3cf6ac6751.jpg: 640x640 3 weeds, 16.4ms
Speed: 5.6ms preprocess, 16.4ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/XYZ/test/images/agri_0_9810_jpeg.r

val: Scanning /content/drive/MyDrive/XYZ/test/labels.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:04<00:00,  1.09s/it]


                   all         50         74      0.908      0.411      0.599      0.408
                  weed         24         42      0.894      0.571      0.685      0.487
                 crops         26         32      0.922       0.25      0.513      0.328
Speed: 0.3ms preprocess, 28.8ms inference, 0.0ms loss, 7.3ms postprocess per image
Results saved to runs/detect/val
Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000
Mean IoU: 0.7889
mAP@[.5:.95]: 0.4079
Custom Evaluation Metric: 0.7039


In [22]:
from ultralytics import YOLO
import os

# Path to the YAML file
# new_data_yaml = '/content/drive/MyDrive/XYZ/train/data_4.yaml'

# Path to save models
save_folder = '/content/drive/MyDrive/XYZ/saved_models'

# Create the folder if it doesn't exist
os.makedirs(save_folder, exist_ok=True)

# Initialize the model
model = YOLO('yolov8s.pt')

# Train the model on your data
model.train(
    data='/content/drive/MyDrive/XYZ/data_4.yaml',                            # New YAML file
    epochs=75,                                     # Number of epochs
    imgsz=640,                                     # Image size
    batch=16,                                      # Batch size
    optimizer='Adam',                              # Optimizer
    lr0=0.005,                                     # Initial learning rate
    weight_decay=0.0005,                           # Weight decay
    augment=True,                                  # Data augmentation
    device='0',                                    # GPU device
    save_period=5,                                 # Save the model every 5 epochs
    save_dir=save_folder                           # Custom save directory
)


Ultralytics 8.3.71 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/XYZ/data_4.yaml, epochs=75, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=5, cache=False, device=0, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, sh

train: Scanning /content/drive/MyDrive/XYZ/train/labels... 980 images, 0 backgrounds, 0 corrupt: 100%|██████████| 980/980 [00:08<00:00, 110.91it/s]


train: New cache created: /content/drive/MyDrive/XYZ/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/drive/MyDrive/XYZ/test/labels.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]


Plotting labels to runs/detect/train7/labels.jpg... 
optimizer: Adam(lr=0.005, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train7
Starting training for 75 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         50         74      0.914      0.438      0.639      0.441



75 epochs completed in 0.686 hours.
Optimizer stripped from runs/detect/train7/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train7/weights/best.pt, 22.5MB

Validating runs/detect/train7/weights/best.pt...
Ultralytics 8.3.71 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.20it/s]


                   all         50         74      0.901       0.45      0.634      0.431
                  weed         24         42      0.895      0.595      0.696      0.493
                 crops         26         32      0.907      0.306      0.572      0.369
Speed: 0.5ms preprocess, 14.0ms inference, 0.0ms loss, 6.2ms postprocess per image
Results saved to runs/detect/train7


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7da8aa36c210>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [23]:
model.save('/content/drive/MyDrive/XYZ/yolov8_trained_model_final_3.pt')

In [ ]:
import os

# Path to the folder containing the label files
labels_folder = '/content/drive/MyDrive/XYZ/train/labels'
images_folder = '/content/drive/MyDrive/XYZ/train/images'

# Function to normalize label coordinates
def normalize_labels(label_file_path, image_width, image_height):
    with open(label_file_path, 'r') as file:
        lines = file.readlines()

    normalized_lines = []
    for line in lines:
        values = line.strip().split()
        if len(values) == 5:
            cls, x_center, y_center, width, height = map(float, values)

            # Normalize the coordinates
            x_center_norm = x_center / image_width
            y_center_norm = y_center / image_height
            width_norm = width / image_width
            height_norm = height / image_height

            # Append normalized label
            normalized_lines.append(f"{cls} {x_center_norm} {y_center_norm} {width_norm} {height_norm}")
        else:
            print(f"Skipping invalid line: {line.strip()}")  # In case there are errors in the label file format

    # Overwrite the label file with normalized values
    with open(label_file_path, 'w') as file:
        file.write("\n".join(normalized_lines))

# Loop through each image and its corresponding label file
for img_file in os.listdir(images_folder):
    if img_file.endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(images_folder, img_file)
        label_file_path = os.path.join(labels_folder, os.path.splitext(img_file)[0] + '.txt')

        if os.path.exists(label_file_path):
            # Get image dimensions (width, height)
            with Image.open(img_path) as img:
                image_width, image_height = img.size

            # Normalize the label file for this image
            normalize_labels(label_file_path, image_width, image_height)

print("Label normalization completed.")


Label normalization completed.


# **Final Model Testing**

In [24]:
from ultralytics import YOLO
import torch
import os
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

# Load the trained YOLOv8 model
model = YOLO('/content/drive/MyDrive/XYZ/yolov8_trained_model_final_3.pt')

# Paths to test images and annotations
test_image_dir = '/content/drive/MyDrive/XYZ/test/images'
test_label_dir = '/content/drive/MyDrive/XYZ/test/labels'

# Function to load ground truth annotations
def load_ground_truth(label_path):
    with open(label_path, 'r') as file:
        lines = file.readlines()
    labels = []
    for line in lines:
        parts = list(map(float, line.strip().split()))
        labels.append(parts)  # [class_id, x_center, y_center, width, height]
    return np.array(labels)

# Evaluation metrics initialization
all_true_classes = []
all_pred_classes = []
all_iou_scores = []

# IoU calculation function
def calculate_iou(box1, box2):
    # Convert from (x_center, y_center, w, h) to (x1, y1, x2, y2)
    def xywh_to_xyxy(box):
        x_c, y_c, w, h = box
        return [x_c - w / 2, y_c - h / 2, x_c + w / 2, y_c + h / 2]

    box1 = xywh_to_xyxy(box1)
    box2 = xywh_to_xyxy(box2)

    # Intersection coordinates
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    # Intersection area
    intersection = max(0, x2 - x1) * max(0, y2 - y1)

    # Union area
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = box1_area + box2_area - intersection

    # IoU
    return intersection / union if union != 0 else 0

# Inference and evaluation loop
for image_file in os.listdir(test_image_dir):
    if image_file.endswith(('.jpg', '.png')):
        image_path = os.path.join(test_image_dir, image_file)
        label_path = os.path.join(test_label_dir, os.path.splitext(image_file)[0] + '.txt')

        # Skip if no corresponding annotation
        if not os.path.exists(label_path):
            continue

        # Inference
        results = model(image_path)
        predictions = results[0].boxes  # Bounding boxes

        # Ground truth
        ground_truths = load_ground_truth(label_path)

        # Get original image dimensions
        image_width, image_height = results[0].orig_shape[1], results[0].orig_shape[0]

        # Matching predictions with ground truth
        for pred in predictions:
            pred_class = int(pred.cls.item())
            pred_box = pred.xywh[0].cpu().numpy()

            # Normalize predicted box coordinates
            pred_box_normalized = pred_box / [image_width, image_height, image_width, image_height]

            iou_scores = [calculate_iou(pred_box_normalized, gt[1:]) for gt in ground_truths]
            best_match_idx = np.argmax(iou_scores)

            all_true_classes.append(int(ground_truths[best_match_idx][0]))
            all_pred_classes.append(pred_class)
            all_iou_scores.append(iou_scores[best_match_idx])

# Metrics Calculation
precision = precision_score(all_true_classes, all_pred_classes, average='weighted')
recall = recall_score(all_true_classes, all_pred_classes, average='weighted')
f1 = f1_score(all_true_classes, all_pred_classes, average='weighted')
mean_iou = np.mean(all_iou_scores)

# Calculate mAP@[.5:.95]
map_results = model.val()
map_50_95 = map_results.box.map  # Mean Average Precision @[.5:.95]

# Custom Evaluation Metric
custom_metric = 0.5 * f1 + 0.5 * map_50_95

# Display Metrics
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Mean IoU: {mean_iou:.4f}")
print(f"mAP@[.5:.95]: {map_50_95:.4f}")
print(f"Custom Evaluation Metric: {custom_metric:.4f}")


image 1/1 /content/drive/MyDrive/XYZ/test/images/agri_0_9135_jpeg.rf.f5afbc71d5606255b3522bfc6feec990.jpg: 640x640 (no detections), 35.1ms
Speed: 13.7ms preprocess, 35.1ms inference, 5.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/XYZ/test/images/agri_0_8991_jpeg.rf.e46eb3e83c49f7b83fd8b49300ff6bf6.jpg: 640x640 (no detections), 24.2ms
Speed: 4.7ms preprocess, 24.2ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/XYZ/test/images/agri_0_9116_jpeg.rf.812b125d18a8e7b60a363dc022354c92.jpg: 640x640 1 crops, 28.6ms
Speed: 11.0ms preprocess, 28.6ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/XYZ/test/images/agri_0_9424_jpeg.rf.f758222e980e6d4265312a3cf6ac6751.jpg: 640x640 2 weeds, 36.8ms
Speed: 11.7ms preprocess, 36.8ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/XYZ/test/images/agri_0_9810_jpe

val: Scanning /content/drive/MyDrive/XYZ/test/labels.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.30it/s]


                   all         50         74      0.675      0.591      0.705      0.472
                  weed         24         42      0.684      0.619      0.747      0.529
                 crops         26         32      0.667      0.562      0.664      0.415
Speed: 0.5ms preprocess, 11.6ms inference, 0.0ms loss, 6.7ms postprocess per image
Results saved to runs/detect/val2
Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000
Mean IoU: 0.8166
mAP@[.5:.95]: 0.4720
Custom Evaluation Metric: 0.7360


# **Final Accuracy is 0.7360**

In [ ]:
import os

# Paths to the directories
unlabeled_images_folder = '/content/drive/MyDrive/XYZ/unlabeled'  # Folder with unlabeled images
train_images_folder = '/content/drive/MyDrive/XYZ/train/images'  # Folder with labeled and pseudo-labeled images

# Get the list of image filenames in both directories
unlabeled_images = set(os.listdir(unlabeled_images_folder))
train_images = set(os.listdir(train_images_folder))

# Find common images between the two directories
common_images = unlabeled_images.intersection(train_images)

# Delete the common images from the unlabeled folder
for img in common_images:
    img_path = os.path.join(unlabeled_images_folder, img)
    try:
        os.remove(img_path)  # Remove the image
        print(f"Deleted {img}")
    except Exception as e:
        print(f"Error deleting {img}: {e}")


Deleted agri_0_7869_jpeg.rf.62f456d467f89f88d7ed78cd99d8e717.jpg
Deleted agri_0_7579_jpeg.rf.a78c42e8f6083518060c6f4f4894b041.jpg
Deleted agri_0_1198_jpeg.rf.8d19f5ccf408dc7633f3298305d688bb.jpg
Deleted agri_0_9891_jpeg.rf.d82f204a3ace7c1d25fcf0861b577eb5.jpg
Deleted agri_0_3869_jpeg.rf.990582a1b9af10b63192f4259baad3ee.jpg
Deleted agri_0_7119_jpeg.rf.06d300499ac10661adb2e96515a91466.jpg
Deleted agri_0_2530_jpeg.rf.15d5fcb891dec627c5b29753e260126f.jpg
Deleted agri_0_2082_jpeg.rf.5545840a67ba34baaa133dce1ecc0a64.jpg
Deleted agri_0_1494_jpeg.rf.55507b6425e2c5887806d8f0fdd8020a.jpg
Deleted agri_0_4016_jpeg.rf.854ec02fc9a990216b409f15a3552a83.jpg
Deleted agri_0_8265_jpeg.rf.275db4e9faf55985a8c6311a3cbeb1c4.jpg
Deleted agri_0_752_jpeg.rf.eb1fbd53735aaac7dd84c988422bc4ef.jpg
Deleted agri_0_3398_jpeg.rf.4f6a160155562bb5ed1a592c9670fb87.jpg
Deleted agri_0_6419_jpeg.rf.16821a112451511ae2ddac4c56a8755a.jpg
Deleted agri_0_1731_jpeg.rf.46d900a124073c3a8ca60ed34caa7130.jpg
Deleted agri_0_4394_jpeg.r